In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

# 讀取數據
train_normal = np.load('Data/train_normal.npy')
train_outer_break = np.load('Data/train_outer_break.npy')
train_inner_break = np.load('Data/train_inner_break.npy')

# 創建標籤
labels_normal = np.zeros(len(train_normal))
labels_outer_break = np.ones(len(train_outer_break))
labels_inner_break = np.full(len(train_inner_break), 2)

# 合併數據和標籤
X_train = np.concatenate((train_normal, train_outer_break, train_inner_break), axis=0)
y_train = np.concatenate((labels_normal, labels_outer_break, labels_inner_break), axis=0)
print(X_train.shape)
print(y_train.shape)

X_train_tensor = torch.tensor(X_train, dtype=torch.float).unsqueeze(1)  # Adding channel dimension
y_train_tensor = torch.tensor(y_train, dtype=torch.long)  # Assuming classification task

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
batch_size = 1  # Adjust the batch size according to your needs and hardware capabilities

# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)



(750, 10240)
(750,)


In [19]:
class VibrationCNN(nn.Module):
    def __init__(self):
        super(VibrationCNN, self).__init__()
        # Assuming 1D Convolutional layers for sequence data
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=4, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv1d(in_channels=4, out_channels=1, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        # The size here might need adjustment based on the size after convolutions and pooling
        # Calculating the size: As input is 10240, after 3 pooling layers, the size is 10240/(2^3) = 1280
        self.fc1 = nn.Linear(1 * 1280, 20)
        self.fc2 = nn.Linear(20, 3)

    def forward(self, x):
        # x shape is [batch_size, channels, sequence_length]
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        
        x = F.relu(self.conv2(x))
        x = self.pool(x)

        # Adjust the number of pool operations according to your model architecture
        # An additional pool operation as an example
        x = self.pool(x)
        
        # Flatten for the fully connected layer
        x = x.view(-1, 1 * 1280)  # Adjust the flattening size accordingly
        
        x = F.relu(self.fc1(x))
        x = self.fc2(x)  # No activation, as nn.CrossEntropyLoss() includes softmax
        
        return x


In [20]:
import torch.optim as optim
import base_func 
model_list = {
    "VibrationCNN" : lambda: VibrationCNN(),
}
model_name = "VibrationCNN"
loaders = base_func.get_dataloaders(train_dataset, 0.6, 0.5, 64)
phases = ['train','val']
base_func.train(model_list, "VibrationCNN", loaders, phases)

--------- INDEX checking ---------
Original: [0, 1, 2, 3, 4]
Shuffled: [19, 8, 330, 277, 343]
--------- INDEX shuffled ---------

Total number of samples: 750 datapoints
Number of train samples: 8 batches/ 450 datapoints
Number of val samples: 3 batches/ 150 datapoints
Number of test samples: 3 batches/ 150 datapoints

Training model: VibrationCNN
Total parameters: 25,728
Learning rate=0.001


100%|██████████| 8/8 [00:00<00:00, 100.03it/s]


Epoch [1/25], phase: train, samples: 450, Loss: 0.0195, Top-1 Accuracy: 36.44%


100%|██████████| 3/3 [00:00<00:00, 300.10it/s]


Epoch [1/25], phase: val, samples: 150, Loss: 0.0218, Top-1 Accuracy: 34.00%


...

100%|██████████| 8/8 [00:00<00:00, 400.13it/s]


Epoch [25/25], phase: train, samples: 450, Loss: 0.0044, Top-1 Accuracy: 100.00%


100%|██████████| 3/3 [00:00<?, ?it/s]

Epoch [25/25], phase: val, samples: 150, Loss: 0.0038, Top-1 Accuracy: 100.00%
Elapsed time: 0.7897355556488037 seconds


VibrationCNN(
  (conv1): Conv1d(1, 4, kernel_size=(5,), stride=(1,), padding=(2,))
  (conv2): Conv1d(4, 1, kernel_size=(5,), stride=(1,), padding=(2,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1280, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=3, bias=True)
)